In this notebook we will train a [Logistic Regression model](https://en.wikipedia.org/wiki/Logistic_regression) to distinguish between feature vectors corresponding to legitimate transactions and fraudulent transactions. 

Logistic Regression is a classic statistical method used for binary classification, meaning that there are 2 possible classes: In our case these are _legitimate_ and _fraudulent_ transactions.

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_parquet("fraud-cleaned-sample.parquet")

# Train/test split

We're using time-series data, so we'll split our data set based on time.

In [ ]:
first = df['timestamp'].min()
last = df['timestamp'].max()
cutoff = first + ((last - first) * 0.7)

df = df.sample(frac=0.1).copy()

train = df[df['timestamp'] <= cutoff]
test = df[df['timestamp'] > cutoff]

We also load in the feature engineering pipeline which we developed in [notebook 2](02-feature-engineering.ipynb).

In [ ]:
import cloudpickle as cp
feature_pipeline = cp.load(open('feature_pipeline.sav', 'rb'))

## Dealing with Imbalanced Classes

When the training data set contains unequal representation from each of your classes we say we are dealing with _imbalanced classes_. In our data set only approximately 2% of the samples are fraudulent, and the remaining 98% are legitimate. Thus we have imbalanced classes. 

This causes problems for a few reasons:
1. a model which classified all new transactions as 'legitimate' would be correct 98% of the time. This high accuracy can trick you into thinking that your model is working well, despite it just returning 'legitimate' for every sample it sees. 
2. even if your model tries to learn patterns in the data, it may struggle to learn frmo the 'fraudulent' data since there simply isn't enough of it.


There are afew approaches we could take to tackling the problem, but the two we are going to use today are: 
1. use metrics which are more informative that if we just counted how many times the model makes a correct classification. 
2. weight the samples by the inverse of the frequency of their label. These weights will be passed into the logistic regression model, and used to ensure that the model is penalized proportionally to this weight for making a missclassification when it is training. 


In this next cell we compute the weights for each of the data lables. 

In [ ]:
fraud_frequency = train[train["label"] == "fraud"]["timestamp"].count() / train["timestamp"].count()
train.loc[train["label"] == "legitimate", "weights"] = fraud_frequency
train.loc[train["label"] == "fraud", "weights"] = (1 - fraud_frequency)


## Training the model

We're now ready to train our Logistic Regression model. We pass the model the feature vectors (generated using our `feature_pipeline` from the previous notebook) and the weights we computed above.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)

svecs = feature_pipeline.fit_transform(train)
lr.fit(svecs, train["label"], sample_weight=train["weights"])

## Model Validation 

We now need to check how well the model performs on data it wasn't trained on. We use the model we just trained to make predictions for the data in our _test_ set, and compare those predictions to the truth. 



In [ ]:
from sklearn.metrics import classification_report

predictions = lr.predict(feature_pipeline.fit_transform(test))
print(classification_report(test.label.values, predictions))


The report shows that the model is performing okay, but is much better at identifying legitimate transactions than fraudulent ones. 

We can visualise these accuracy of classifications in a confusion matrix.

In [ ]:
from mlworkflows import plot
df, chart = plot.binary_confusion_matrix(test["label"], predictions)
chart

Viewing the raw counts emphasises that the model misclassifies a lot of 'fraudulent' transactions as 'legitimate'. 

In [ ]:
df

# Save the model as a pipeline stage

In [ ]:
from mlworkflows import util
util.serialize_to(lr, "lr.sav")